# Names : 미국 연도별 출생아 이름 데이터

In [ ]:
names2010 = pd.read_csv(r"C:\Users\Administrator\Downloads\names\names\yob2010.txt",
                       names = ['name','sex','birth'])
names2010.info()
names2010.groupby('sex').sum()

In [ ]:
pieces = []
for year in range(1880,2011):
    path = r"C:\Users\Administrator\Downloads\names\names\yob%d.txt" % year
    df = pd.read_csv(path, names = ['name','sex','birth'])
    df['year'] = year
    pieces.append(df)
    
# pd.concat(pieces).loc[0]  
## concat 했을 떄 여러개의 인덱스가 발생되는 문제가 있기 떄문에

names = pd.concat(pieces, ignore_index=True) # ignore index 해줘야한다.
names

In [ ]:
# birth를 데이터, 행 인덱스를 year, 열을 sex로 구성해서 pivot table 만들기
total_birth = names.pivot_table(values = 'birth', index='year', columns='sex', aggfunc="sum")
# aggfunc 집계함수 default = mean
total_birth

In [ ]:
total_birth.plot(title='Total births')

In [ ]:
def add_prop(group):
    group['prop'] = group.birth / group.birth.sum()
    return group
names = names.groupby(['year','sex']).apply(add_prop)

names

In [ ]:
np.allclose # 특정 값에 가까운지 확인하는 것
np.allclose(names.groupby(['year','sex']).prop.sum(),1)

In [ ]:
# 각 연도별, 성별에 따른 이름 빈도수가 가장
# 높은 이름을 1000개씩 추출
def get_top_1000(group):
    return group.sort_index(by='birth', ascending=False)[:1000]

grouped = names.groupby(['year','sex'])
top1000 = grouped.apply(get_top_1000)

girls = top1000[top1000.sex=='F']
boys = top1000[top1000.sex=='M']

top1000.reset_index(inplace=True, drop=True)
top1000

In [ ]:
# 2010 남자 아이들의 상위 50%의 이름들의 개수
df = boys[boys.year==2010]
prop_cumsum = df.sort_index(by='prop', ascending=False).prop.cumsum()
prop_cumsum[prop_cumsum <= 0.5]

df.sort_index(by='prop', ascending=False)[df.sort_index(by='prop', ascending=False).prop.cumsum() < 0.5]

prop_cumsum.values.searchsorted(0.5)
# 누적합이 0.5인 지점까지 이름 종류의 개수

In [ ]:
num_half_ratio_names = []
for i in range(1880,2011):
    df = boys[boys.year==i]
    prop_cumsum = df.sort_index(by='prop', ascending=False).prop.cumsum()
    num_half_ratio_names.append(prop_cumsum.values.searchsorted(0.5))

In [ ]:
def get_qc(group, q=0.5):
    group = group.sort_index(by='prop', ascending=False)
    return group.prop.cumsum().values.searchsorted(q)
res = names.groupby(['year','sex']).apply(get_qc)
res = res.unstack('sex')
res
res.plot()

In [ ]:
top1000

In [ ]:
1900, 1950, 2010년
성별로 구분
이름을 구성하는 알파벳 문자 비율

1900년 남자아이들 이름 

가장 많이 사용된 알파벳 문자는?

남자아이들, 여자아이들 년도별로 다 나눠서
각 비율들 까지

In [ ]:
names.head()

In [ ]:
names.pivot_table(index = ['year','name'])

In [2]:
import pandas as pd
import numpy as np

pieces = []
for year in range(1880,2011):
    path = r"C:\Users\Administrator\Downloads\names\names\yob%d.txt" % year
    df = pd.read_csv(path, names = ['name','sex','birth'])
    df['year'] = year
    pieces.append(df)
    
# pd.concat(pieces).loc[0]  
## concat 했을 떄 여러개의 인덱스가 발생되는 문제가 있기 떄문에

names = pd.concat(pieces, ignore_index=True) # ignore index 해줘야한다.
names

,name,sex,birth,year
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880
3,Elizabeth,F,1939,1880
4,Minnie,F,1746,1880
...,...,...,...,...
1690779,Zymaire,M,5,2010
1690780,Zyonne,M,5,2010
1690781,Zyquarius,M,5,2010
1690782,Zyran,M,5,2010


In [55]:
def alphas(data, year, sex):
    data = data[data.sex == sex]
    data = data[(data.year == year)].reset_index()
    
    alpha = {}
    for a in list('abcdefghijklmnopqrstuvwxyz'):
        alpha[a] = 0
        for i in range(len(data)):
            alpha[a] += data.loc[i,'birth'] * data.loc[i,'name'].lower().count(a)
    return pd.DataFrame(pd.Series(alpha), columns=[year])

alpha1900 = alphas(names, 1900, "M")
alpha1950 = alphas(names, 1950, "M")
alpha2010 = alphas(names, 2010, "M")

In [92]:
alpha_1 = pd.merge(alpha1900, alpha1950, left_index=True, right_index=True)
alpha_2 = pd.merge(alpha_1, alpha2010, left_index=True, right_index=True)
total = alpha_2.copy()
total

,1900,1950,2010
a,81762,1051887,1554280
b,11776,172561,204049
c,23948,318208,384734
d,29166,551746,461567
e,105748,1141946,1110083
f,10355,73497,33779
g,15695,188931,134705
h,43656,496685,430917
i,49662,621435,741094
j,28043,282568,335540


In [93]:
total.loc['sum'] = total.sum(axis=0)

In [106]:
total['1900_ratio'] = round(total[1900]/total.loc['sum',1900],3)
total['1950_ratio'] = round(total[1950]/total.loc['sum',1950],3)
total['2010_ratio'] = round(total[2010]/total.loc['sum',2010],3)
total_male = total.copy()
total_male

,1900,1950,2010,1900_ratio,1950_ratio,2010_ratio
a,81762,1051887,1554280,0.097,0.103,0.141
b,11776,172561,204049,0.014,0.017,0.018
c,23948,318208,384734,0.029,0.031,0.035
d,29166,551746,461567,0.035,0.054,0.042
e,105748,1141946,1110083,0.126,0.112,0.100
f,10355,73497,33779,0.012,0.007,0.003
g,15695,188931,134705,0.019,0.018,0.012
h,43656,496685,430917,0.052,0.049,0.039
i,49662,621435,741094,0.059,0.061,0.067
j,28043,282568,335540,0.033,0.028,0.030


In [107]:
alpha1900 = alphas(names, 1900, "F")
alpha1950 = alphas(names, 1950, "F")
alpha2010 = alphas(names, 2010, "F")

alpha_1 = pd.merge(alpha1900, alpha1950, left_index=True, right_index=True)
alpha_2 = pd.merge(alpha_1, alpha2010, left_index=True, right_index=True)
total = alpha_2.copy()
total.loc['sum'] = total.sum(axis=0)
total['1900_ratio_F'] = round(total[1900]/total.loc['sum',1900],3)
total['1950_ratio_F'] = round(total[1950]/total.loc['sum',1950],3)
total['2010_ratio_F'] = round(total[2010]/total.loc['sum',2010],3)
total_female = total.copy()
total_female

,1900,1950,2010,1900_ratio_F,1950_ratio_F,2010_ratio_F
a,238264,1637224,2068129,0.143,0.161,0.193
b,27026,243263,188031,0.016,0.024,0.018
c,40606,375266,210578,0.024,0.037,0.020
d,42581,391657,247245,0.026,0.039,0.023
e,269827,1143386,1206827,0.162,0.113,0.113
f,13236,21792,28710,0.008,0.002,0.003
g,27597,113508,125086,0.017,0.011,0.012
h,58876,349266,343187,0.035,0.034,0.032
i,146019,779417,1050072,0.088,0.077,0.098
j,11380,182528,125225,0.007,0.018,0.012


In [ ]:
total_male